In [1]:
import torch
from transformers import AlbertModel, AlbertTokenizer
import re
import os
import requests
from tqdm.auto import tqdm

In [2]:

tokenizer = AlbertTokenizer.from_pretrained("Rostlab/prot_albert", do_lower_case=False)

Downloading:   0%|          | 0.00/238k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/505 [00:00<?, ?B/s]

In [3]:
model = AlbertModel.from_pretrained("Rostlab/prot_albert")

Downloading:   0%|          | 0.00/897M [00:00<?, ?B/s]

Some weights of the model checkpoint at Rostlab/prot_albert were not used when initializing AlbertModel: ['predictions.dense.bias', 'predictions.LayerNorm.weight', 'predictions.dense.weight', 'predictions.LayerNorm.bias', 'sop_classifier.classifier.weight', 'predictions.bias', 'sop_classifier.classifier.bias', 'predictions.decoder.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model = model.eval()

In [5]:
sequences_Example = ["A E T C Z A O","S K T Z P"]

In [6]:
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

In [7]:
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, pad_to_max_length=True)

/home/anwer/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2302: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [8]:
input_ids = torch.tensor(ids['input_ids']).to(device)
attention_mask = torch.tensor(ids['attention_mask']).to(device)

In [12]:

with torch.no_grad():
    embedding = model(input_ids=input_ids,attention_mask=attention_mask)[0]

In [13]:
embedding = embedding.cpu().numpy()
features = [] 
for seq_num in range(len(embedding)):
    seq_len = (attention_mask[seq_num] == 1).sum()
    seq_emd = embedding[seq_num][1:seq_len-1]
    features.append(seq_emd)

In [14]:
features

[array([[ 0.0504114 ,  0.01671049, -0.06041686, ...,  0.03879312,
          0.06742979, -0.02243829],
        [ 0.04481406,  0.02007729, -0.06041646, ...,  0.01449897,
          0.07357281, -0.02993694],
        [ 0.04433526,  0.02113996, -0.05988096, ...,  0.01485694,
          0.07245751, -0.02932365],
        ...,
        [ 0.04479378,  0.01773688, -0.06211098, ...,  0.01833749,
          0.07087199, -0.02901261],
        [ 0.03966043,  0.04050677, -0.01892166, ...,  0.02172128,
          0.06361923, -0.0444499 ],
        [ 0.04467373,  0.028899  , -0.06304027, ...,  0.00830533,
          0.08290324, -0.02485269]], dtype=float32),
 array([[-0.03166272,  0.01354348, -0.02900332, ..., -0.11964942,
         -0.11535285,  0.07926516],
        [-0.04238589,  0.01891197, -0.02533977, ..., -0.12101973,
         -0.10938682,  0.0965146 ],
        [-0.04246924,  0.02128097, -0.02400075, ..., -0.12410869,
         -0.10866602,  0.10307008],
        [-0.0435378 ,  0.02157612, -0.0225751 , ...,